Now we will do text classification to classify the books in it's categories
because normal vector seach does not give us as accurate results, we can consider it as a baseline result


In [2]:
#just to see we have475 categories
import pandas as pd
books = pd.read_csv('books_cleaned.csv')

books["categories"].value_counts()

categories
Fiction                                    2075
Juvenile Fiction                            377
Biography & Autobiography                   302
History                                     203
Literary Criticism                          121
                                           ... 
Amis, Kingsley                                1
Dangerously mentally ill                      1
Anger                                         1
African Americans in radio broadcasting       1
Otherland (Imaginary place)                   1
Name: count, Length: 475, dtype: int64

In [12]:
#to limit CATEGORIES we limit number of books in each category to greater than 50
books["categories"].value_counts().reset_index().query('count>50')

,categories,count
0,Fiction,2075
1,Juvenile Fiction,377
2,Biography & Autobiography,302
3,History,203
4,Literary Criticism,121
5,Comics & Graphic Novels,116
6,Philosophy,112
7,Religion,112
8,Drama,81
9,Science,55


In [18]:
#now we create a category mapping to classify catefgories as nonfiction and fiction
category_mapping = {"Biography & Autobiography":"Nonfiction",
               "History":"Nonfiction",
                    "Literary Criticism":"Nonfiction",
                    "Philosophy":"Nonfiction",
                    "Religion":"Nonfiction",
                    "Comics & Graphic Novels":"Fiction",
                    "Drama":"Fiction",
                    "Juvenile Nonfiction":"Children's Nonfiction",
                    "Science":"Nonfiction",
                    "Poetry":"Fiction",
}

books["simple_categories"] = books["categories"].map(category_mapping)


In [23]:
books.simple_categories.isna().sum()

np.int64(3885)

In [25]:
#using a hugging face for a zero shot classifcation
from transformers import pipeline

fiction_categories = ["Fiction", "Nonfiction"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# pipe = pipeline("Zero-Shot-Classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [41]:
#the description of the book which we want to classify
sequence = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[1]
sequence

"At last...the public hearing she was denied...These essays reveal keen powers of analysis applied to some of the most obdurate problems that bedevil electoral politics. Anyone who cares about the mechanisms of democracy should be engaged by her tough-minded explorations. It doesn't matter where you think you stand: it's all here, to argue or agree with. -- Henry Louis Gates, Jr. Lani Guinier's fascinating book is a prophetic intervention into a public conversation we desperately need to rejuvenate. There is no doubt that her powerful voice will produce good consequences for our nation and world. -- Cornel West, Author of Race Matters Intriguing and desperately needed... -- The San Francisco Chronicle"

In [34]:
#in the output, we get the labels and it's probability
pipe(sequence, fiction_categories)

{'sequence': "Claudia Corvette. From her tousled bedroom hair to her name–all the porn stars in this world take their names from supermodels and sports cars–she is adult entertainment's prototypical femme fatale. Her life is the collision of countless troubled–childhood cliches and grown–up wet dreams, projected onto her as surely as her videos project their blue light onto lonely men around the world. From its first panel, How to Make Money Like a Porn Star draws the reader into the dark world of girls like Claudia, the men who fantasize about them, and the monsters who control them. In the hands of Rolling Stone writer Neil Strauss and illustrator Bernard Chang, this adult graphic novel weaves together black humor and blacker reality. Like all great American stories, it features humble beginnings, life–changing tragedy, stripping, abuse, implants, fame, addiction, bigger implants, abduction, gunplay, downfall, and even bigger implants. Not to mention a thousand shades of latex and L'

In [35]:
# better processing of output
import numpy as np
def generate_output(sequence, categories):
     predictions = pipe(sequence, categories)
     max_index = np.argmax(predictions['scores'])
     max_label = predictions['labels'][max_index]
     return max_label


In [38]:
from tqdm import tqdm #tells us how much time will it take

actual_cats = []

predicted_cats = []

# create lists of actual values and predicted values for fiction catefgory
for i in tqdm(range(0,247)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_output(sequence, fiction_categories)]
    actual_cats += ["Fiction"]

100%|██████████| 247/247 [05:29<00:00,  1.34s/it]


In [42]:
for i in tqdm(range(0,300)):
    sequence = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_output(sequence, fiction_categories)]
    actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [08:41<00:00,  1.74s/it]


In [43]:
predictions_df = pd.DataFrame({"actual_categories" : actual_cats,"predicted_categories":predicted_cats})

In [44]:
predictions_df

,actual_categories,predicted_categories
0,Fiction,Nonfiction
1,Fiction,Fiction
2,Fiction,Nonfiction
3,Fiction,Fiction
4,Fiction,Fiction
...,...,...
545,Nonfiction,Nonfiction
546,Nonfiction,Nonfiction
547,Nonfiction,Fiction
548,Nonfiction,Nonfiction


In [45]:
predictions_df["correction_prediction"] = (np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0))

In [47]:
# Accuracy
predictions_df["correction_prediction"].sum() / len(predictions_df)

np.float64(0.7181818181818181)

In [48]:
isbns = []
predicted_cats = []
missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13","description"]].reset_index(drop=True)

In [49]:
for i in tqdm(range(0,len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_output(sequence, fiction_categories)]
    isbns += [missing_cats["isbn13"][i]]

100%|██████████| 3885/3885 [1:27:47<00:00,  1.36s/it]  


In [50]:
missing_predicted_df = pd.DataFrame({"isbn13":isbns, "predicted_categories":predicted_cats})

In [51]:
missing_predicted_df

,isbn13,predicted_categories
0,9780002005883,Fiction
1,9780002261982,Fiction
2,9780006178736,Fiction
3,9780006280897,Nonfiction
4,9780006280934,Nonfiction
...,...,...
3880,9788125026600,Nonfiction
3881,9788171565641,Fiction
3882,9788172235222,Fiction
3883,9788173031014,Nonfiction


In [56]:
books =pd.merge(books, missing_predicted_df, on="isbn13", how="left")
books['simple_categories'] =np.where(books['simple_categories'].isna(), books['predicted_categories'],books['simple_categories'])
books = books.drop(columns = ['predicted_categories'])

In [57]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title and subtitle,tagged_description,simple_categories,predicted_categories_x,predicted_categories_y
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,Fiction,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,Fiction,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,Fiction,Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,Nonfiction,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,Nonfiction,Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,Fiction,Fiction
5084,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,Nonfiction,Nonfiction
5085,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,Fiction,Fiction
5086,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,NaN,NaN


In [58]:
books.to_csv("books_with_categories.csv", index=False)